<a href="https://colab.research.google.com/github/jwang44/Try-colabing-in-colab/blob/main/orthopedic_patients_new_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ECSE 551 Mini-project 1
*Group 10: Junhao Wang, Yinan Zhou, and Ruilin Ji*

This notebook is dedicated for the **Orthopedic patients dataset**, including the model, cross validation, and various experiment. 

## Start here

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd "/content/drive/My Drive/"

/content/drive/My Drive


In [3]:
import numpy as np
import pandas as pd
import scipy.stats
import statistics

## Orthopedic Patients Dataset

In [4]:
# generate new feature by multiplication and normalize
def newfeature(x,y):
  z=x*y
  norz=scipy.stats.zscore(z, axis=0, ddof=0, nan_policy='propagate')
  return norz

In [5]:
# convert csv to dataframe
df = pd.read_csv('orthopedic_patients.csv')
original_data = df.to_numpy()

In [6]:
# normalize feature
NorData = scipy.stats.zscore(df.iloc[:,:-1], axis=0, ddof=0, nan_policy='propagate')
# normalized dataset
NorDataset = np.column_stack((NorData, df['Class']))
# np.savetxt('normalized_orthopedic_patients.csv', NorPatientData, delimiter=',')

New feature

In [58]:
# new feature
NewF1 = newfeature(df.pelvic_incidence, df.sacral_slope) # new feature by multiplying two high-correlation feature
NewF2 = df.iloc[:,0]             # initialize new feature 2 using 1st feature
n_row = np.shape(original_data)[0]
NewF = np.zeros(n_row)            # initialize new features using 0s
for col in range(6):
  # new feature 2: multiplying all features
  if col>0:
    NewF2 = newfeature(NewF2,df.iloc[:,col])
  # new feature 3-8: square all columns
  new = newfeature(df.iloc[:,col],df.iloc[:,col]) # square feature
  NewF = np.column_stack((NewF,new))

NewF = np.delete(NewF,0,1)
# print(NewF2)
# print(np.shape(NewF))
# normalized dataset with new feature
NorDatasetNew = np.column_stack((NorData,NewF1,NewF2,NewF,df['Class']))

## The model

In [61]:
# sigmoid function
def sigmoid(a):
  return 1/(1+np.exp(-a))

class Logistic_regression():
  def __init__(self):#,X_train,y_train,learning_rate,X_test,y_test):
    pass
    
  # training
  def fit(self,X_train,y_train,learning_rate):
    #n,m = np.shape(self.X_train)
    n,m = np.shape(X_train)  
    wk = np.ones([m+1,1]) # wk weights, initialized with 1
    wk1 = np.zeros([m+1,1])# wk+1 weights,initialized with 0          
    itrnum = 500       # max number of iterations 
    e = 0.001
    der = 0
    for k in range(0,itrnum):
      for i in range(0,n):
        #xi = self.X_train[i].T
        xi = X_train[i].T
        x0 = np.array([1])
        xi = np.concatenate((xi, x0),axis = 0)
        #yi = self.y_train[i]
        yi = y_train[i]
        der = der-xi*(yi-sigmoid(np.matmul(wk[:,0].T,xi))) # take derivative w.r.t w
      #wk1[:,0] = wk[:,0]-self.learning_rate*der       # update rule
      wk1[:,0] = wk[:,0]-learning_rate*der       # update rule
      if (np.linalg.norm(wk1[:,0]-wk[:,0]))**2<e:         
        break 
      else:
        wk = wk1.copy()
    return wk1
  
  # validation
  def predict(self,w,X_test):
    #n,m = np.shape(self.X_test)
    n,m = np.shape(X_test)   
    y_predict = np.zeros([n,1])
    for i in range(0,n):
      #xi = self.X_test[i].T
      xi = X_test[i].T
      x0 = np.array([1])
      xi = np.concatenate((xi, x0),axis = 0)
      p1 = sigmoid(np.matmul(w.T,xi)) # calculate probabilities p(y=1|x)
      # covert probabilities to 0 or 1 by thresholding at 0.5
      if p1>=0.5:
        y_predict[i] = 1
      else:
        y_predict[i] = 0
    return y_predict

  # evaluate accuracy
  def Accu_eval(self,y_test,y_predict):
    #y_predict = self.predict(X_test)
    n,j = np.shape(y_predict)
    TP = 0;FP = 0;TN = 0;FN = 0
    # count TP,TN,FP,FN in validation set
    '''for i in range(n):
      if  self.y_test[i]==1 and y_predict[i]==1:
        TP = TP+1
      elif self.y_test[i]==0 and y_predict[i]==0:
        TN = TN+1
      elif self.y_test[i]==0 and y_predict[i]==1:
        FP = FP+1
      elif self.y_test[i]==1 and y_predict[i]==0:
        FN = FN+1'''
    for i in range(n):
      if  y_test[i]==1 and y_predict[i]==1:
        TP = TP+1
      elif y_test[i]==0 and y_predict[i]==0:
        TN = TN+1
      elif y_test[i]==0 and y_predict[i]==1:
        FP = FP+1
      elif y_test[i]==1 and y_predict[i]==0:
        FN = FN+1    
    accuracy = (TP+TN)/(TP+TN+FP+FN)
    # precision = TP/(TP+FP)
    # recall = TP/(TP+FN)
    # F = 2*precision*recall/(precision+recall)
    # specificity = TN/(FP+TN)
    # FPR = FP/(FP+TN)
    print("accuracy:",accuracy)
    # print("precision:",precision)
    # print("recall:",recall)
    # print("F:",F)
    # print("specificity:",specificity)
    # print("False Positive Rate:",FPR)
    print("")
    return accuracy
    

In [43]:
# figure out which feature is of the most importance
model = Logistic_regression()
np.random.shuffle(NorDataset)
X = NorDataset[:, :-1]  # features
y = NorDataset[:, -1]   # labels
w = model.fit(X,y,learning_rate=0.001)
print(w)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until


[[  2.44739104]
 [  0.88561015]
 [ -0.19250647]
 [  2.9123171 ]
 [  4.89809744]
 [-16.30535573]
 [ -3.98992869]]


## Cross validation

In [26]:
class Cross_validation():
  def __init__(self, k):
    # k: k-fold
    self.k = k

  def prepare_data(self, data):
    # data: np array converted from csv
    np.random.shuffle(data)
    X = data[:, :-1]  # features
    y = data[:, -1]   # labels

    # split data into k equal segments, assign them to train and test later
    Xs = np.array_split(X, self.k, axis=0)
    ys = np.array_split(y, self.k, axis=0)
    return Xs, ys

  def get_accuracy(self, Xs, ys, lr):
    accu_trains = []
    accu_tests = []
    for i in range(self.k):
      X_cv = Xs[:] # X_cross_validation
      y_cv = ys[:] # y_cross_validation

      X_test = X_cv.pop(i)
      y_test = y_cv.pop(i)

      X_train = np.concatenate(X_cv)
      y_train = np.concatenate(y_cv)

      model = Logistic_regression()
      w = model.fit(X_train, y_train, lr)

      print("----------FOLD ", i+1, "----------")

      print("----Train----")
      y_predict_train = model.predict(w, X_train)
      accu_train = model.Accu_eval(y_train, y_predict_train)
      accu_trains.append(accu_train)

      print("----Validation----")
      y_predict_test = model.predict(w, X_test)
      accu_test = model.Accu_eval(y_test, y_predict_test)
      accu_tests.append(accu_test)

    return np.mean(accu_trains), np.mean(accu_tests)


## Experiment with different learning rates

In [ ]:
lrs = np.logspace(-5, -1, 10) # different learning rates to try
cv = Cross_validation(10) # 10-fold cross-validation
Xs, ys = cv.prepare_data(NorDataset)
for lr in lrs:
  print("---------------LEARNING RATE: ", lr, "---------------")
  accu_train_avg, accu_val_avg = cv.get_accuracy(Xs, ys, lr)
  print("---------------TRAIN AVERAGE ACCURACY", accu_train_avg, "---------------")
  print("---------------VALIDATION AVERAGE ACCURACY", accu_val_avg, "---------------")
  print("\n-------------------------------------------------------------------------------\n")


## Experiment with different features

During the experiment on different learning rates, we found that the best learning rate is **0.0129**, so we use this learning rate for our experiment with different feature selections. 

In [40]:
lr = 0.0129
cv = Cross_validation(10) # 10-fold cross-validation
Xs, ys = cv.prepare_data(NorDataset)
print("----------Using normalized features, without new features----------")
accu_avg_train, accu_avg_val = cv.get_accuracy(Xs, ys, lr)
print("----------AVERAGE ACCURACY", accu_avg_val, "----------")
print("\n---------------------------------------------------------------------")


----------Using normalized features, without new features----------


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until


----------FOLD  1 ----------
----Train----
accuracy: 0.8494623655913979

----Validation----
accuracy: 0.8064516129032258

----------FOLD  2 ----------
----Train----
accuracy: 0.8207885304659498

----Validation----
accuracy: 0.7741935483870968

----------FOLD  3 ----------
----Train----
accuracy: 0.8387096774193549

----Validation----
accuracy: 0.8387096774193549

----------FOLD  4 ----------
----Train----
accuracy: 0.8136200716845878

----Validation----
accuracy: 0.7741935483870968

----------FOLD  5 ----------
----Train----
accuracy: 0.8602150537634409

----Validation----
accuracy: 0.7741935483870968

----------FOLD  6 ----------
----Train----
accuracy: 0.8315412186379928

----Validation----
accuracy: 0.8709677419354839

----------FOLD  7 ----------
----Train----
accuracy: 0.8458781362007168

----Validation----
accuracy: 0.8387096774193549

----------FOLD  8 ----------
----Train----
accuracy: 0.8387096774193549

----Validation----
accuracy: 0.9032258064516129

----------FOLD  9 ------

In [ ]:
lr = 0.0129
cv = Cross_validation(10) # 10-fold cross-validation
Xs, ys = cv.prepare_data(original_data)
print("----------Using normalized features, with new features----------")
accu_avg_train, accu_avg_val = cv.get_accuracy(Xs, ys, lr)
print("----------AVERAGE ACCURACY", accu_avg_val, "----------")
print("\n---------------------------------------------------------------------")


In [62]:
lr = 0.0129
cv = Cross_validation(10) # 10-fold cross-validation
Xs, ys = cv.prepare_data(NorDatasetNew)
print("----------Using normalized features, with new features----------")
accu_avg_train, accu_avg_val = cv.get_accuracy(Xs, ys, lr)
print("----------AVERAGE ACCURACY: train-", accu_avg_train," vs. validation-",accu_avg_val, "----------")
print("\n---------------------------------------------------------------------")


----------Using normalized features, with new features----------


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until


----------FOLD  1 ----------
----Train----
accuracy: 0.8387096774193549

----Validation----
accuracy: 0.9032258064516129

----------FOLD  2 ----------
----Train----
accuracy: 0.8566308243727598

----Validation----
accuracy: 0.8387096774193549

----------FOLD  3 ----------
----Train----
accuracy: 0.8745519713261649

----Validation----
accuracy: 0.6774193548387096

----------FOLD  4 ----------
----Train----
accuracy: 0.8422939068100358

----Validation----
accuracy: 0.9032258064516129

----------FOLD  5 ----------
----Train----
accuracy: 0.8637992831541219

----Validation----
accuracy: 0.8387096774193549

----------FOLD  6 ----------
----Train----
accuracy: 0.8602150537634409

----Validation----
accuracy: 0.8064516129032258

----------FOLD  7 ----------
----Train----
accuracy: 0.8387096774193549

----Validation----
accuracy: 0.9032258064516129

----------FOLD  8 ----------
----Train----
accuracy: 0.8566308243727598

----Validation----
accuracy: 0.8709677419354839

----------FOLD  9 ------

## Measure the run time
See whether the model converges faster on normalized data than on original data. This is measured by training the model and time it. This part stands on its own, and is not related to any of the above process

This model class is almost the same as the one before. The only difference is that this model **does not have early stopping**, which can interfere with our timing. Also, this model is only for timing the training process, so it only has a 'fit' method

**Should we run this? Should we have early stopping or not?**

In [59]:
class Logistic_regression():
  def __init__(self):
    pass
    
  # training
  def fit(self,X_train,y_train,learning_rate):
    #n,m = np.shape(self.X_train)
    n,m = np.shape(X_train)  
    wk = np.ones([m+1,1]) # wk weights, initialized with 1
    wk1 = np.zeros([m+1,1])# wk+1 weights,initialized with 0          
    itrnum = 500       # max number of iterations 
    e = 0.001
    der = 0
    for k in range(0,itrnum):
      for i in range(0,n):
        #xi = self.X_train[i].T
        xi = X_train[i].T
        x0 = np.array([1])
        xi = np.concatenate((xi, x0),axis = 0)
        #yi = self.y_train[i]
        yi = y_train[i]
        der = der-xi*(yi-sigmoid(np.matmul(wk[:,0].T,xi))) # take derivative w.r.t w
      #wk1[:,0] = wk[:,0]-self.learning_rate*der       # update rule
      wk1[:,0] = wk[:,0]-learning_rate*der       # update rule
      # if (np.linalg.norm(wk1[:,0]-wk[:,0]))**2<e:         
      #   break 
      # else:
      wk = wk1.copy()
    return wk1

###Using the original dataset

In [ ]:
model = Logistic_regression()
np.random.shuffle(original_data)
X = original_data[:, :-1]  # features
y = original_data[:, -1]   # labels

In [ ]:
%%timeit
w = model.fit(X, y, learning_rate=0.00001)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until


1 loop, best of 3: 2.02 s per loop


In [ ]:
%%timeit
w = model.fit(X, y, learning_rate=0.001)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until


1 loop, best of 3: 2.08 s per loop


In [ ]:
%%timeit
w = model.fit(X, y, learning_rate=0.01)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until


1 loop, best of 3: 2.11 s per loop


###Using normalized features

In [ ]:
model = Logistic_regression()
np.random.shuffle(NorDataset)
X = NorDataset[:, :-1]  # features
y = NorDataset[:, -1]   # labels

In [ ]:
%%timeit
w = model.fit(X, y, learning_rate=0.00001)

100 loops, best of 3: 3.57 ms per loop


In [ ]:
%%timeit
w = model.fit(X, y, learning_rate=0.001)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until


1 loop, best of 3: 1.84 s per loop


In [ ]:
%%timeit
w = model.fit(X, y, learning_rate=0.01)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until


1 loop, best of 3: 1.79 s per loop
